In [ ]:
import json
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Step 1: Load data
with open('your_data_file.json', 'r') as file:
    data = json.load(file)

# Convert data to a sorted list of tuples (date, price)
sorted_data = sorted((pd.to_datetime(date), float(price)) for date, price in data.items())

# Separate dates and prices
dates, prices = zip(*sorted_data)

# Step 2: Preprocess data
prices = np.array(prices).reshape(-1, 1)

# Normalize prices
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_prices = scaler.fit_transform(prices)

# Step 3: Split the data
train_data, test_data = train_test_split(scaled_prices, test_size=0.2, shuffle=False)

# Step 4: Prepare data for LSTM
def create_dataset(data, time_step=1):
    X, Y = [], []
    for i in range(len(data) - time_step - 1):
        a = data[i:(i + time_step), 0]
        X.append(a)
        Y.append(data[i + time_step, 0])
    return np.array(X), np.array(Y)

time_step = 100
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

# Reshape input to be [samples, time steps, features]
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Step 5: Build the LSTM model
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(50, return_sequences=True, input_shape=(time_step, 1)),
    tf.keras.layers.LSTM(50, return_sequences=False),
    tf.keras.layers.Dense(25),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')

# Step 6: Train the model
model.fit(X_train, y_train, batch_size=1, epochs=1)

# Step 7: Evaluate the model
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

# Transform back to original form
train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)

# Calculate RMSE performance metrics
train_rmse = np.sqrt(np.mean(train_predict - y_train)**2)
test_rmse = np.sqrt(np.mean(test_predict - y_test)**2)

print(f'Train RMSE: {train_rmse}, Test RMSE: {test_rmse}')
